# What We'll Cover

<div class="alert alert-info" style="margin-bottom: 16px; padding: 12px; border-left: 4px solid #428bca; background-color: #f8f9fa;">

**Backtests of simple MACD momentum Bitcoin strategies**
* Inspect Bitcoin.py for class definitions
* BenchmarkBitcoinStrategy class is a simple benchmark buy and hold strategy
* BacktestBitcoinStrategy_01 class is a MACD strategy as starting point
* BacktestBitcoinStrategy_02 class is MACD + ATR based volatility filter
* BacktestBitcoinStrategy_03 class is MACD + Moving averages short and long
      
**Regular Optimization**
* Single train/test split
* Initial validation
      

**Walkforward Optimization**
* Two different configurations  
* Robust time-series testing

**Comparrison of performances accross optimizations**
* Out of sample performaces
* Overfitting

# Imports

In [ ]:
%matplotlib qt 
import pandas as pd
import backtrader_manager as bm

In [ ]:
# the classes defining the backtests can be found in Bitcoin.py
from Bitcoin import (
    BenchmarkBitcoinStrategy,
    BacktestBitcoinStrategy_01, # only macd indicator
    BacktestBitcoinStrategy_02, # macd and atr indicators
    BacktestBitcoinStrategy_03, # macd and ma short/long indicators
    ) 

# Parameter configurations

</div><div style="border: 1px solid #f0ad4e; padding: 12px; border-radius: 4px; background-color: #fcf8e3; margin-bottom: 16px;">
Time-Saving Mode

The constrained ranges sigificantly reduce computation time.

To run full optimization:
* Uncomment complete parameter ranges
* Restart kernel
* Re-run notebook
</div> 

In [ ]:
# Minimal ranges for quick testing
parameters_macd = {
    'macd_slow': [26, 40, 60], 
    'macd_fast': [12, 24], 
    'macd_signal': [9] 
}

parameters_ma_short_long = {
    'ma_short': [25, 50],
    'ma_long': [100, 150, 200]
}

parameters_atr = {
    'atr_period': [7, 14],
    'atr_threshold': [0.02],
}

# Backtest Collection 

In [ ]:
# Path to pandas ohlc data
csv_datas_path = '../../datas/Bitcoin' # in this case relative path, absolute path also possible
# Name of collection
name = 'Main'

# Instantiate new collection
main = bm.BacktestCollection(name=name, csv_datas_path=csv_datas_path)

<span style="background-color: #f9e79f; padding: 2px 4px; border-radius: 3px;">**✅ Automatic Saving**</span>  
• Collection is saved after each backtest  
• Includes all cerebro result objects  

<span style="background-color: #d5f5e3; padding: 2px 4px; border-radius: 3px;">**🔁 Recovery Option**</span>  
• If PC crashed, simply re-run the cell above 
• Restores last saved state  


# Regular Optimization

## Train

In [ ]:
# Instatiate a new BacktestInputCollection
name = 'single_opt' 

single_opt = bm.BacktestInputCollection(
    name, # name that decribes type of optimization  
    max_warmup = 200, # maximum bars reserved for any indicator warmup phase prior backtest start
    train_perc=0.7, # defines train/test split
)

In [ ]:
# We add it to main
# Now the datafeeds are accesible and the train-test periods are calculated
main.add_backtest_input_collection(single_opt)

In [ ]:
# Create backtest inputs for our strategies
# we start with Benchmark
backtest_class = BenchmarkBitcoinStrategy
strategy_parameters = None # this strategy doesnt need any parameters

single_opt.create_backtest_inputs(backtest_class=backtest_class, strategy_parameters=strategy_parameters)

In [ ]:
# Create inupts for the remaining backtest classes
single_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_01, strategy_parameters={**parameters_macd})
single_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_02, strategy_parameters={**parameters_macd, **parameters_atr})
single_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_03, strategy_parameters={**parameters_macd, **parameters_ma_short_long})

In [ ]:
# Create new backtests 
main.create_new_backtests(single_opt)

In [ ]:
# Run new backtests
main.run_backtests()

In [ ]:
# summary dataframe contain all completed backtests with meta data, metrics and parameters
main.summary

In [ ]:
# same datafame but in groups 
main.summarize_in_groups()

In [ ]:
# same datafame but in groups, sorted_by sharperatio and showing strategy parameters columns
summary_ranked = main.summarize_in_groups(period_key='train', sort_by=('sharperatio', 'mean'), show_parameters=True).head(5)
summary_ranked

In [ ]:
# We select first row as optimum to be tested out of sample
optimum_single_opt = summary_ranked.iloc[0].name
optimum_single_opt

## Test

In [ ]:
# We create the input for the test period 
single_opt.create_backtest_inputs(strategy_parameters='strat_03-31', period_keys=('test',))
single_opt.create_backtest_inputs(strategy_parameters='buy_hold-1', period_keys=('test',))

In [ ]:
# Create new backtests 
main.create_new_backtests(single_opt)

In [ ]:
# Run new backtests
main.run_backtests()

## Results

In [ ]:
# Create quantstats reports


In [ ]:
# Summary dataframe
main.summarize_in_groups(parameter_id=['strat_03-31', 'buy_hold-1'])

- strategy has a higher sharperatio in test period than in train period
- but: strategy underperformed compared to benchmark in test period

# Walkforward Optimization 1

## Train

In [ ]:
# Instatiate a new BacktestInputCollection
name = 'wfo_1_opt' 

wfo_1_opt = bm.BacktestInputCollection(
    name, # name that decribes type of optimization  
    max_warmup = 200, # maximum bars reserved for any indicator warmup phase prior backtest start
    train_perc=0.7, # defines train/test split
)

In [ ]:
# We add it to main
main.add_backtest_input_collection(wfo_1_opt)

In [ ]:
# For a walk forward optimisation we first decide which window size to use
# Herefor we plot our data usings bars instead of timestamps
wfo_1_opt.plot_train_test_periods(use_dates=False)

In [ ]:
# We try out differnet window sizes, train/test splits and walkforward steps and recalculate the train_test_periods
# You can comment out the values and play arround with the settings
wfo_1_opt.window = 750
wfo_1_opt.train_perc = 0.7
wfo_1_opt.walkforward_step = wfo_1_opt.test_window

In [ ]:
wfo_1_opt.train_window

In [ ]:
wfo_1_opt.test_window

In [ ]:
wfo_1_opt.calc_train_test_periods()

In [ ]:
# The following workflow is the same as with the reguar optimization

# Create backtest inputs for our strategies
wfo_1_opt.create_backtest_inputs(backtest_class=BenchmarkBitcoinStrategy, strategy_parameters=None)
wfo_1_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_01, strategy_parameters={**parameters_macd})
wfo_1_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_02, strategy_parameters={**parameters_macd, **parameters_atr})
wfo_1_opt.create_backtest_inputs(backtest_class=BacktestBitcoinStrategy_03, strategy_parameters={**parameters_macd, **parameters_ma_short_long})

# Create and run backtests
main.create_new_backtests(wfo_1_opt)
main.run_backtests()

In [ ]:
# Benchmark performance
main.summarize_in_groups(input_collection_name='wfo_1_opt', backtest_class='buy_hold')

In [ ]:
# Strategy performaces sorted by mean sharperatio and showing parameters
summary_ranked = main.summarize_in_groups(input_collection_name='wfo_1_opt', period_key='train', sort_by=('sharperatio', 'mean'), show_parameters=True).head(5)
summary_ranked

In [ ]:
optimum_wfo_1_opt = summary_ranked.iloc[0].name
optimum_wfo_1_opt

## Test

In [ ]:
# These are all the backtes of our selcted optimum and benchmark in the train period
summary_filtered = main.summarize_filter_by(
    input_collection_name=optimum_wfo_1_opt[0], 
    period_key=optimum_wfo_1_opt[1], 
    parameter_id=[optimum_wfo_1_opt[3], 'buy_hold-1'])
summary_filtered

In [ ]:
# We could now create our inputs as done in the regular optimization on by one
# But a quicker way is by using the following method
# Exact same inputs are created with the exeption priod_key that is set to 'test'
wfo_1_opt.create_cloned_inputs(summary_filtered, period_keys=('test',))

In [ ]:
len(wfo_1_opt.backtest_inputs)

In [ ]:
# Create and run backtests
main.create_new_backtests(wfo_1_opt)
main.run_backtests()

## Results

In [ ]:
# Summary dataframe
main.summarize_in_groups(
    input_collection_name=optimum_wfo_1_opt[0], 
    parameter_id=[optimum_wfo_1_opt[3], 'buy_hold-1'])

- strategy has a higher sharperatio in test period than in train period
- strategy outperformed benchmark also in test period

# Walkforward Optimization 2

In [ ]:
# Add second wfo with smaller windows more iterations

## Train

## Test

## Result

# Results Comparison

In [ ]:
optimum_single_opt

In [ ]:
optimum_wfo_1_opt

In [ ]:
single_opt = main.get_input_collection('single_opt')
single_opt.plot_train_test_periods()

In [ ]:
wfo_1_opt = main.get_input_collection('wfo_1_opt')
wfo_1_opt.plot_train_test_periods()

In [ ]:
main.summarize_in_groups(
    input_collection_name=optimum_single_opt[0],
    parameter_id=[optimum_single_opt[3], 'buy_hold-1'],
    show_parameters=True
)

In [ ]:
main.summarize_in_groups(
    input_collection_name=optimum_wfo_1_opt[0],
    parameter_id=[optimum_wfo_1_opt[3], 'buy_hold-1'],
    show_parameters=True
)

In [ ]:
# Some kind of resumee